# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several ML models to play against, and in doing so explore:
1. Deploying models trained in your choice of framework to Clipper with a few lines of code using Clipper's model deployers.
2. Easily update or roll back models that have already been deployed in live applications.
3. Use machine learning and Clipper to do something fun!

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and deploying an initial (bad) model
#### 2. Training a better model
#### 3. Deploying the new and improved model to the live Pong application

## Part 1: Starting Clipper and deploying your first Pong AI model

In [ ]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from sklearn import linear_model
import requests
from IPython.display import Markdown, display

this_ip = requests.get('http://ip.42.pl/raw').text
this_ip

The first step is to start Clipper and deploy your first model: one that randomly guesses which direction to move the paddle.

The cell below will start Clipper running in Docker containers. You can run the `docker ps` shell comamnd to see the Clipper Docker containers. By this point, the Clipper Docker images should already be downloaded on the server. But if you decided to do the exercises out of order and are starting Clipper for the first time, this command may take a few minutes while it downloads the Docker images.

In [ ]:
# Start Clipper. This command assumes that Docker is already running.
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for the random policy at http://localhost:1337/pong/predict

In [ ]:
def random_predict(xs):
    '''
    Deploy a policy that returns a random choice from 0, 1, or 2.
    Remember that Clipper requires the output of the predict function to
    be a list of string objects.
    '''
    action = None # TODO randomly choose an action from the choices 0, 1, 2
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000, )

**Solution:**

```python
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)
```

Now that you have a model deployed, let's see how it works! Run the cell below to start the little web app that will serve the Pong game.

In [ ]:
from subprocess import Popen, PIPE

server_proc = Popen([
    "python", 
    "pong-server/pong-server.py", 
    "localhost:1337", 
    this_ip,
    "pong_server.log"], stdout=PIPE)

display(Markdown(f"""
This is your link to pong game:
http://{this_ip}:4000/pong/
"""))

Congratulations! We have deployed our first Pong AI to Clipper! Let's see how well it works by **clicking the link above and pressing 1 to start the game.**

> *If you need to stop the server for any reason, you can run the following command to stop it:*
>```py
server_proc.terminate()
```

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, you will use [imitation learning](https://katefvision.github.io/katefSlides/immitation_learning_I_katef.pdf). Imitation learning is often used for reinforcement learning, but in this case we are going to use it to train a simple classifier.

Play the game again, this time with 2 human players, and we will collect data on how you play the game to train the model.

First, pair up with one of the people sitting next to you to play against. Then go back to your pong game and **press 2 to start a 2-player game.** After you have played a few games on your computer, switch to your partner's computer and play on their instance so you both have training data.

### After you've finished playing Pong

Now that you have some training data, it's time to train a model. First, run the cell below to clean the data and format it for Scikit-Learn's LogisticRegression model.

In [ ]:
df_data = pd.read_csv('out.csv')
df_data.columns = ["label","paddle_y","ball_x","ball_y","ball_dx","ball_dy","x_prev","y_prev"]

def convert_label(label):
    """Convert labels into numeric values"""
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0

df_data['label'] = df_data['label'].apply(convert_label)
df_data.loc[:, "paddle_y":"y_prev"] = df_data.loc[:, "paddle_y":"y_prev"]/500.0

df_data.head()

In [ ]:
df_data.size

You will use the data to train a scikit-learn Logistic Regression model, just as you did in the previous exercise. You can read more about the particular model [in the documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [ ]:
labels = df_data['label']
training_data= df_data.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)

## Part 3: Deploying the new model to a live application

Now that you have a better model, you can deploy that model to Clipper. Once the system detects there is a new version of the model, it will automatically start routing requests to the new version.

![update_model](notebook-images/pong_update_model.png)

In [ ]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

# TODO: Fill in the missing values below to deploy a version 2 of the pong model container. 
# It takes in inputs of type double and uses the predict function defined above.

py_deployer.deploy_python_closure(clipper_conn, name="", version="", input_type="", func="",
                                  pkgs_to_install=["numpy","scipy", "pandas", "sklearn"])


**Solution:**

```python
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

# TODO: Fill in the missing values below to deploy a version 2 of the pong model container. 
# It takes in inputs of type double and uses the predict function defined above.

py_deployer.deploy_python_closure(clipper_conn, name="pong", version=2, input_type="doubles", func=predict, pkgs_to_install=["numpy","scipy", "pandas", "sklearn"])
```

Go to pong link we showed above , press 1 to start a new game against the AI, and notice how the game AI has improved with your new model!

### Next steps

This is the end of the Clipper tutorial. If you have finished early, you can continue trying to improve the Pong model. Scikit-Learn has several different classifiers you can experiment with. For example, you might see how a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) or an [SVM](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) perform. 

In [ ]:
# server_proc.terminate()

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model to imitate your own Pong playing behavior.
3. Deployed this new version of model to a live application without any downtime.

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation



Run the cell below to stop clipper:

In [ ]:
clipper_conn.stop_all() 
